In [1]:
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import folium
import requests
from tqdm import tqdm
from collections import deque
import matplotlib.cm as cm
import matplotlib.colors as colors

# Load previous data in pt.2

In [3]:
TorontoData = pd.read_hdf("TorontoData.hdf")
TorontoData.head()

,Borough,Neighbourhood,Latitude,Longitude
Postcode,,,,
M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [4]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(TorontoData['Borough'].unique()),
        TorontoData.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


# Toronto map & the data collected

In [5]:
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [6]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

neighborhoods = TorontoData

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'],
                                           neighborhoods['Longitude'],
                                           neighborhoods['Borough'],
                                           neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

# Foursquare access
#### I had trouble figuring out my own foursquare id so I used others found on the Internet.

In [18]:
CLIENT_ID = '0MJA3NYYG3U2ZY1LTZN2OYEHS3Y3WVSON2GBSO3IL4EDYVIR' # your Foursquare ID
CLIENT_SECRET = 'WGWSAF2TKVUQPE3PD0N3EOITFVBY5EYP1VCZI3BMUG0ROUS5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [19]:
# funtion takes the names and locations of the neighborhoods in Toronto and obtains the 100 top venues around.
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT = 100):
    
    venues_list=[]
    for name, lat, lng in tqdm(zip(names, latitudes, longitudes), total = names.size):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
Toronto_venues = getNearbyVenues(TorontoData.Neighbourhood,
                            TorontoData.Latitude,
                            TorontoData.Longitude)

100%|██████████| 103/103 [00:50<00:00,  2.94it/s]


In [21]:
print(Toronto_venues.shape)
Toronto_venues.head()

(2241, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


In [22]:
Toronto_venues.groupby("Neighborhood").Venue.count().sort_values(ascending=False).head()

Neighborhood
Adelaide, King, Richmond                     100
First Canadian Place, Underground city       100
Ryerson, Garden District                     100
St. James Town                               100
Chinatown, Grange Park, Kensington Market    100
Name: Venue, dtype: int64

In [23]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 280 uniques categories.


In [24]:
Toronto_OHE = pd.get_dummies(Toronto_venues["Venue Category"],
                             prefix = "",
                             prefix_sep = "")

Toronto_OHE["Neighborhood"] = Toronto_venues["Neighborhood"]


nindex = list(Toronto_OHE.columns).index("Neighborhood")
cols = deque(Toronto_OHE.columns)
cols.rotate(-nindex)
cols = list(cols)
Toronto_OHE = Toronto_OHE[cols]

Toronto_OHE.head()

,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,...,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,Nail Salon
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
Toronto_OHE.shape

(2241, 280)

In [26]:
Toronto_grouped = Toronto_OHE.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head()

,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,...,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,Nail Salon
0,"Adelaide, King, Richmond",0.01,0.0,0.01,0.01,0.01,0.0,0.0,0.0,0.0,...,0.0,0.01,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0
1,Agincourt,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
4,"Alderwood, Long Branch",0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
Toronto_grouped.shape

(100, 280)

In [28]:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,Bar,Breakfast Spot,Asian Restaurant,Gym,Restaurant,American Restaurant
1,Agincourt,Lounge,Sandwich Place,Chinese Restaurant,Breakfast Spot,Clothing Store,Nail Salon,Aquarium,Airport Service,Airport Terminal,American Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Coffee Shop,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Coffee Shop,Video Store,Beer Store,Pizza Place,Pharmacy,Fast Food Restaurant,Sandwich Place,Fried Chicken Joint,Arts & Crafts Store
4,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Sandwich Place,Gym,Pharmacy,Athletics & Sports,Pub,Skating Rink,Auto Garage,Asian Restaurant


# Use PCA to reduce noise & improve clustering efficiency

In [30]:
pca = PCA(.95)
Toronto_grouped_clustering = pca.fit_transform(Toronto_grouped.drop('Neighborhood', 1))
Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

The dimensionality reduction has kept only a reduced number of the total dimensions.

In [31]:
Toronto_grouped_clustering.shape

(100, 279)

With reduced dimensional dataset,I carry out the KMeans clustering.

In [32]:
# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[0:10])
print(kmeans.labels_.shape)

[0 0 2 4 4 0 0 0 0 0]
(100,)


#### a dataframe that containes the neighborhood, the location and the cluster information, together with the top 10 venues

In [33]:

Toronto_grouped["Cluster Labels"] = kmeans.labels_

# add clustering labels
Toronto_combined = TorontoData.merge(Toronto_grouped, left_on = "Neighbourhood", right_on = "Neighborhood", how = "outer")

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_combined = Toronto_combined.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_combined["Cluster Labels"] = Toronto_combined["Cluster Labels"].fillna(5).astype("int")

Toronto_combined.head() # check the last columns!

,Borough,Neighbourhood,Latitude,Longitude,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,"Rouge, Malvern",43.806686,-79.194353,"Rouge, Malvern",0.0,0.0,0.0,0.0,0.0,...,Fast Food Restaurant,Nail Salon,Aquarium,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Airport Food Court
1,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,"Highland Creek, Rouge Hill, Port Union",0.0,0.0,0.0,0.0,0.0,...,Bar,Nail Salon,Art Gallery,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arts & Crafts Store
2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,"Guildwood, Morningside, West Hill",0.0,0.0,0.0,0.0,0.0,...,Electronics Store,Rental Car Location,Intersection,Pizza Place,Medical Center,Breakfast Spot,Mexican Restaurant,Airport Lounge,Airport Service,Airport Terminal
3,Scarborough,Woburn,43.770992,-79.216917,Woburn,0.0,0.0,0.0,0.0,0.0,...,Coffee Shop,Insurance Office,Korean Restaurant,Mexican Restaurant,Nail Salon,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant
4,Scarborough,Cedarbrae,43.773136,-79.239476,Cedarbrae,0.0,0.0,0.0,0.0,0.0,...,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Fried Chicken Joint,Hakka Restaurant,American Restaurant,Airport Gate,Airport Lounge


In [34]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

kclusters = kclusters + 1

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_combined['Latitude'],
                                  Toronto_combined['Longitude'],
                                  Toronto_combined['Neighborhood'],
                                  Toronto_combined['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Clusters

## Cluster 1

In [35]:
Toronto_combined.loc[Toronto_combined['Cluster Labels'] == 0, 
                     "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Bar,Nail Salon,Art Gallery,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arts & Crafts Store
2,Electronics Store,Rental Car Location,Intersection,Pizza Place,Medical Center,Breakfast Spot,Mexican Restaurant,Airport Lounge,Airport Service,Airport Terminal
3,Coffee Shop,Insurance Office,Korean Restaurant,Mexican Restaurant,Nail Salon,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant
4,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Fried Chicken Joint,Hakka Restaurant,American Restaurant,Airport Gate,Airport Lounge
5,Playground,Smoke Shop,Art Gallery,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arts & Crafts Store


## Cluster 2

In [36]:
Toronto_combined.loc[Toronto_combined['Cluster Labels'] == 1, 
                     "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Fast Food Restaurant,Nail Salon,Aquarium,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Airport Food Court


## Cluster 3

In [37]:
Toronto_combined.loc[Toronto_combined['Cluster Labels'] == 2, 
                     "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Park,Playground,Coffee Shop,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium
23,Park,Bank,Convenience Store,Airport Gate,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery
25,Fast Food Restaurant,Food & Drink Shop,Park,Nail Salon,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop
30,Park,Airport,Construction & Landscaping,Nail Salon,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery
40,Park,Convenience Store,Pizza Place,Coffee Shop,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop


## Cluster 4

In [38]:
Toronto_combined.loc[Toronto_combined['Cluster Labels'] == 3, 
                     "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Baseball Field,Nail Salon,Art Gallery,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arts & Crafts Store
97,Baseball Field,Furniture / Home Store,Nail Salon,Aquarium,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop


## Cluster 5

In [39]:
Toronto_combined.loc[Toronto_combined['Cluster Labels'] == 4, 
                     "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Pizza Place,Fast Food Restaurant,Noodle House,Thai Restaurant,Italian Restaurant,Chinese Restaurant,Bank,Fried Chicken Joint,Antique Shop,Airport Lounge
15,Fast Food Restaurant,Chinese Restaurant,Thrift / Vintage Store,Pharmacy,Noodle House,Breakfast Spot,Grocery Store,Sandwich Place,Pizza Place,Coffee Shop
24,Coffee Shop,Discount Store,Butcher,Pizza Place,Pharmacy,Grocery Store,Antique Shop,Airport Lounge,Airport Service,Airport Terminal
33,Discount Store,Athletics & Sports,Liquor Store,Gym / Fitness Center,Grocery Store,Airport Terminal,Airport Food Court,Airport Gate,Airport Lounge,Airport Service
34,Nail Salon,Coffee Shop,Hockey Arena,Grocery Store,Portuguese Restaurant,Asian Restaurant,Arts & Crafts Store,Art Gallery,Aquarium,Airport Food Court
